# Bibliotecas e funções auxiliares

In [1]:
import numpy as np
import os.path as osp
import pandas as pd
import re

from time import sleep
from datetime import datetime
from tqdm.auto import tqdm

from websearch import WebSearchEngine
from websearch import WebSearchDataManager


In [2]:
def clean_models(models: list):
    stop_words = ['+',  '120hz', '2', '4g', '5g', 'amarelo', 'azul',
                  'basic', 'bluetooth', 'br', 'branco', 'cinza', 'de',
                  'earbuds', 'earphone', 'earphones', 'escuro', 'fone',
                  'gradiente', 'inteligente', 'laranja', 'mi', 'prata',
                  'preto', 'pulseira', 'rosa', 'roxo', 'tela', 'true',
                  'verde', 'wireless', 'wireless']

    pattern = '(([\d]+gb)?[\d]+gb)|(\([\d\w-]+\))|(\d+,\d+)|([+"\/\',’])'
    models = [model.lower() for model in models]
    models = [re.sub(pattern,'',model) for model in models]
    models = list(set([' '.join([token for token in model.split() if token not in stop_words]) for model in models]))
    
    return models

# Carga e preparação dos dados

## Importação dos arquivos

In [3]:
# importar tabela completa do SCH (Dados Abertos)
file_sch = 'datasets/produtos_certificados.zip'
usecols = [0, 1, 2, 3, 11, 12, 13, 14, 15]
dtype = {'Número de Homologação': 'str',
         'CNPJ do Solicitante': 'str'}
df_sch = pd.read_csv(file_sch,sep=';',usecols=usecols,dtype=dtype)

# contar quantidade de números de homologação
df_sch['Quantidade de Número de Homologação']=df_sch[['Número de Homologação','Modelo']].groupby('Número de Homologação').transform('count')
df_sch = df_sch.sort_values(by=['Quantidade de Número de Homologação','Data da Homologação'],ascending=False).reset_index(drop=True)
df_sch.head()

,Data da Homologação,Número de Homologação,Nome do Solicitante,CNPJ do Solicitante,Nome do Fabricante,Modelo,Nome Comercial,Categoria do Produto,Tipo do Produto,Quantidade de Número de Homologação
0,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",W34+,Smartwatch,2,Equipamento de Radiocomunicação de Radiação Re...,319
1,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",W34+,Relógio Inteligente,2,Equipamento de Radiocomunicação de Radiação Re...,319
2,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",HW18,Smartwatch,2,Equipamento de Radiocomunicação de Radiação Re...,319
3,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",HW18,Relógio Inteligente,2,Equipamento de Radiocomunicação de Radiação Re...,319
4,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",HW19,Smartwatch,2,Equipamento de Radiocomunicação de Radiação Re...,319


In [4]:
# importar subset do SCH
file_subset_sch = 'datasets/subset_sch.txt'
dtype = {'Número de Homologação': 'str'}
df_subset_sch = pd.read_csv(file_subset_sch,names=['Número de Homologação'],dtype=dtype)

# merge SCH subset and SCH to join columns "Modelo" and "Nome Comercial"
columns_to_merge = ['Número de Homologação', 'Modelo', 'Nome Comercial']
df_subset_sch=df_subset_sch.merge(df_sch[columns_to_merge])
df_subset_sch = df_subset_sch.fillna('#NULO#')
df_subset_sch = df_subset_sch.drop_duplicates()
df_subset_sch.head()

,Número de Homologação,Modelo,Nome Comercial
0,037242214637,W34+,Smartwatch
1,037242214637,W34+,Relógio Inteligente
2,037242214637,HW18,Smartwatch
3,037242214637,HW18,Relógio Inteligente
4,037242214637,HW19,Smartwatch


In [5]:
# importa lista de EAN da Xiaomi
file_ean_xiaomi = 'datasets/ean_xiaomi.xlsx'
dtype = {'EAN': 'str'}
df_ean_xiaomi = pd.read_excel(file_ean_xiaomi,dtype=dtype)
df_ean_xiaomi.head()

,Código. Referencia,Descrição,EAN
0,CX298VRD,"Smartphone Xiaomi Redmi 9A Tela 6,53"" 2GB/32GB...",7898567779018
1,CX298AZU,"Smartphone Xiaomi Redmi 9A Tela 6,53"" 2GB/32GB...",7898567778998
2,CX298CIN,"Smartphone Xiaomi Redmi 9A Tela 6,53"" 2GB/32GB...",7898567779001
3,CX297VRD,"Smartphone Xiaomi Redmi 9 Tela 6,53"" 4GB/64GB ...",7898567778875
4,CX297ROX,"Smartphone Xiaomi Redmi 9 Tela 6,53"" 4GB/64GB ...",7898567778868


# Pequisas

## Preparação dos dados para gerar itens de pesquisa

In [6]:
ean_to_search = df_ean_xiaomi['EAN'].to_list()
ean_models_to_search = clean_models(df_ean_xiaomi['Descrição'])
sch_to_search = df_subset_sch['Número de Homologação'].unique().tolist()
sch_models_to_search = list(set(df_subset_sch['Modelo'].unique().tolist()+df_subset_sch['Nome Comercial'].unique().tolist()))
all_items_to_search = ean_to_search+ean_models_to_search+sch_to_search+sch_models_to_search
bing_items_to_search = ean_to_search+ean_models_to_search+sch_to_search

# importa resultaos de busca passados
file_search_results = 'datasets/search_results/products_search_results.parquet'
if osp.exists(file_search_results):
    df_search_results = pd.read_parquet(file_search_results)
    google_previous_search = df_search_results[df_search_results['search_provider']=='Google']['original_query'].unique().tolist()
    bing_previous_search = df_search_results[df_search_results['search_provider']=='Bing']['original_query'].unique().tolist()
else:
    google_previous_search = []
    bing_previous_search = []
    
google_items_to_search = [item for item in all_items_to_search if item not in google_previous_search]
bing_items_to_search = [item for item in bing_items_to_search if item not in bing_previous_search]

print('{} items left to search in Google'.format(len(google_items_to_search)))
print('{} items left to search in Bing'.format(len(bing_items_to_search)))

0 items left to search in Google
0 items left to search in Bing


## Objetos de pesquisa e gerenciamento de dados

In [7]:
search_data = WebSearchDataManager()
search_data.load_search_results()
print('{} previous searches results loaded'.format(len(search_data.clean_search_results)))

187401 previous searches results loaded


In [8]:
search_engine = WebSearchEngine()

## Google Search

In [9]:
# a cada 100 consutar verificar se o tempo é inferior a 60 segundo, 
# caso afirmativo, esperar para não ultrapassar a quota do Google
check_quota_step = 100
last_check_timestamp = datetime.now().timestamp()

min_search_time_interval = 0.35

# opcional salvar resultados parcials em dataframe a cada verificação de quota
partial_save_on_check = True

# cota diária de pesquisa do Google é de 10.000 consultas
# atualizar para o valor restante para evitar erros
actual_daily_search_quota = 9950

for count,item in enumerate(tqdm(google_items_to_search[:actual_daily_search_quota])):
    try:
        clean_results,raw_results = search_engine.google_search(item)
        search_data.save_raw_search_results(raw_results)
        search_data.update_search_results(clean_results) 

        # garante um intervalo mínimo de min_search_time_interval para 
        # evitar o estouro da cota de 100 pesquisas por minuto
        # experimentos indicaram que 0.35s/pesquisa é suficiente
        search_time = raw_results['searchInformation']['searchTime']
        if search_time < min_search_time_interval:
            time_to_sleep = min_search_time_interval - search_time
            sleep(time_to_sleep)
    
        if (count + 1) % check_quota_step == 0:
            actual_time = datetime.now().timestamp()
            elapsed_time_since_last_check = actual_time - last_check_timestamp
            print('Elapsed time since last check: {:.2f} seconds'.format(elapsed_time_since_last_check))
    
            last_check_timestamp = actual_time

            if partial_save_on_check:
                # file_to_save = 'products_search_results.parquet.{}'.format(datetime.now().timestamp())
                file_to_save = 'products_search_results.parquet_'
                search_data.save_clean_search_results(file_to_save)
                print('Partial file {} saved on step {}'.format(file_to_save,count+1))
                
    except Exception as ex:
        print('Saved current data... ')
        search_data.save_clean_search_results()
        raise ex

search_data.save_clean_search_results()

0it [00:00, ?it/s]

## Bing Search

In [10]:
# a cada 100 consutar verificar se o tempo é inferior a 60 segundo, 
# caso afirmativo, esperar para não ultrapassar a quota do Google
check_quota_step = 100
last_check_timestamp = datetime.now().timestamp()

min_search_time_interval = 0.35

# opcional salvar resultados parcials em dataframe a cada verificação de quota
partial_save_on_check = True

# cota diária de pesquisa do Google é de 10.000 consultas
# atualizar para o valor restante para evitar erros
actual_daily_search_quota = 9000

for count,item in enumerate(tqdm(bing_items_to_search[:actual_daily_search_quota])):
    try:
        clean_results,raw_results = search_engine.bing_search(item)
        search_data.save_raw_search_results(raw_results)
        search_data.update_search_results(clean_results) 

        # cota de chamadas no Bing é de 3 por segundo e 1.000 chamadas a cada 30 dias 
            
        if (count + 1) % check_quota_step == 0:
            actual_time = datetime.now().timestamp()
            elapsed_time_since_last_check = actual_time - last_check_timestamp
            print('Elapsed time since last check: {:.2f} seconds'.format(elapsed_time_since_last_check))
    
            last_check_timestamp = actual_time

            if partial_save_on_check:
                # file_to_save = 'products_search_results.parquet.{}'.format(datetime.now().timestamp())
                file_to_save = 'products_search_results.parquet_'
                search_data.save_clean_search_results(file_to_save)
                print('Partial file {} saved on step {}'.format(file_to_save,count+1))
                
    except Exception as ex:
        print('Saved current data... ')
        search_data.save_clean_search_results()
        raise ex

search_data.save_clean_search_results()

0it [00:00, ?it/s]